## Estruturação de um Pipeline Modular para Avaliação de Modelos

### 1. Objetivo do Pipeline Modular

O objetivo principal desta refatoração é transicionar da análise exploratória e modelagem sequencial (presente no *notebook* de prototipagem) para um ***framework* de experimentação robusto e generalizável**.

Este *pipeline* modular visa formalizar o pré-processamento e a avaliação de múltiplos algoritmos de aprendizado de máquina de forma estruturada, programática e metodologicamente sólida. O resultado final é uma plataforma que facilita a avaliação comparativa e imparcial do desempenho de diferentes estimadores, garantindo a reprodutibilidade dos resultados.

---

### 2. Motivações para a Refatoração

A transição de um *script* de análise manual para um *pipeline* modularizado é motivada por diversas limitações metodológicas e operacionais da abordagem anterior:

* **Mitigação de *Data Leakage* (Vazamento de Dados):** Esta é a principal motivação. No *workflow* manual, o pré-processamento (ex: `StandardScaler`) é frequentemente aplicado ao conjunto de dados *antes* da divisão de validação cruzada. Isso "contamina" os *folds* de treino com informações estatísticas (média, desvio padrão) do conjunto de validação. O `sklearn.Pipeline` resolve isso ao garantir que o `fit` dos transformadores ocorra *exclusivamente* dentro dos dados de treino de cada *fold* da validação cruzada (`cross_validate`), simulando corretamente o desempenho em dados nunca vistos.
* **Modularidade e Escalabilidade:** O *framework* desacopla a lógica de pré-processamento da lógica de modelagem. O `ColumnTransformer` torna-se um artefato de engenharia de *features* reutilizável, e os modelos são definidos em um dicionário (`models_to_compare`), permitindo a adição ou remoção de algoritmos (ex: `RandomForest`, `LogisticRegression`) sem reescrever o fluxo de dados.
* **Consistência e Comparabilidade:** Ao garantir que todos os modelos recebam *exatamente* os mesmos dados de entrada (pós-processamento) e sejam avaliados sob os mesmos *folds* de validação cruzada, o *pipeline* elimina variáveis de confusão. Isso permite uma comparação direta e justa ("apples-to-apples") da eficácia algorítmica intrínseca de cada modelo.
* **Generalização de Estratégias Ordinais:** A criação de um *wrapper* customizado (`OrdinalRegressorWrapper`) formaliza a abordagem de regressão ordinal. Isso permite que *qualquer* regressor compatível com `sklearn` (ex: `XGBRegressor`, `LinearRegression`) seja avaliado nesta tarefa sem a necessidade de etapas manuais de pós-processamento (como `np.round` e `np.clip`), que são propensas a erros.

---

### 3. Arquitetura da Solução

A arquitetura da solução é centrada na interação de dois componentes principais do `scikit-learn`: `ColumnTransformer` e `Pipeline`.

1.  **Identificação de Atributos:** Primeiramente, as *features* de entrada (`X_train`) foram segregadas programaticamente em dois subconjuntos mutuamente exclusivos:
    * `numeric_features` (ex: `sales`, `ano`, `mes`)
    * `categorical_features` (ex: `region`, `city`, `sub-category`)

2.  **Transformadores Específicos:**
    * **Numéricos:** Para o subconjunto numérico, foi definido um `numeric_transformer`, que consiste em um `Pipeline` simples contendo o `StandardScaler`. Esta etapa normaliza os dados (média zero, variância unitária), um pré-requisito para a performance ótima de modelos lineares (como `LogisticRegression`) e outros algoritmos sensíveis à escala.
    * **Categóricos:** Para o subconjunto categórico, foi definido um `categorical_transformer`. Este aplica o `OneHotEncoder` (OHE), que transforma variáveis nominais em uma representação binária esparsa (vetorização *one-hot*). Esta é uma etapa **crítica** que torna *features* não numéricas compatíveis com a vasta maioria dos algoritmos do `sklearn`. O parâmetro `handle_unknown='ignore'` foi usado para garantir robustez, permitindo que o modelo ignore (não gere erro) se categorias não vistas no treino aparecerem na predição.

3.  **Composição (ColumnTransformer):** O `ColumnTransformer` atua como o orquestrador do pré-processamento. Ele é configurado para aplicar os transformadores `num` e `cat` aos seus respectivos subconjuntos de colunas. O parâmetro `remainder='drop'` foi utilizado para descartar explicitamente quaisquer colunas não especificadas nas listas (ex: `customer_id`), prevenindo o vazamento de dados ou erros de tipo nos estimadores.

4.  **Pipeline Final e Avaliação:** O `ColumnTransformer` (nomeado `preprocessor`) é encadeado como a primeira etapa de um `sklearn.Pipeline` final. O estimador (modelo) do dicionário `models_to_compare` é inserido como a segunda etapa (`model`). A função `cross_validate` é então invocada sobre este *pipeline* completo, automatizando o processo de:
    * Dividir os dados (CV=5).
    * Para cada *fold*, aplicar `fit_transform` do `preprocessor` nos dados de treino.
    * Aplicar `transform` do `preprocessor` nos dados de validação.
    * Treinar (`fit`) o modelo nos dados de treino processados.
    * Avaliar (`score`) o modelo nos dados de validação processados.

---

### 4. A Exclusão Estratégica do CatBoost

O CatBoost não foi incluído neste *pipeline* comparativo por uma **razão metodológica fundamental**: sua arquitetura de processamento de *features* é incompatível com a abordagem de pré-processamento padronizada (via `OneHotEncoder`) adotada para os demais modelos.

O principal diferencial competitivo do CatBoost, e a razão de seu forte desempenho no *notebook* exploratório, é seu **manuseio nativo de variáveis categóricas**. Ele não utiliza OHE, mas sim algoritmos internos de estatística-alvo (como *Target-based Statistics* e *Ordered Boosting*), que são aplicados através do objeto `catboost.Pool` ou da detecção automática de *strings*.

Incluir o CatBoost neste *pipeline* significaria alimentá-lo com dados já transformados pelo `OneHotEncoder`. Isso teria duas consequências negativas:

1.  **Anularia a Vantagem Competitiva:** O modelo seria privado de seu principal mecanismo de otimização de *features*, sendo forçado a tratar as *features* categóricas da mesma forma que um `RandomForest`.
2.  **Degradaria a Eficiência:** O modelo teria que processar um *dataset* com milhares de colunas binárias esparsas (resultado do OHE em `city` ou `state`), para o qual não é otimizado, em vez de um *dataset* enxuto com *features* categóricas brutas.

Portanto, uma comparação justa exigiria um *pipeline* de avaliação separado para o CatBoost, onde as *features* numéricas são escaladas, mas as categóricas são passadas como *strings* brutas (`remainder='passthrough'`), permitindo que o algoritmo aplique sua lógica interna. A comparação, neste caso, não seria apenas entre algoritmos, mas entre *estratégias de processamento* distintas.

In [9]:
from pathlib import Path

import pandas as pd
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import cross_validate
from sklearn.metrics import make_scorer, accuracy_score, mean_absolute_error

# Para a Abordagem B (Regressão Ordinal)
from sklearn.base import BaseEstimator, ClassifierMixin, RegressorMixin

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [2]:
# --- Carregamento e Limpeza (Etapas Faltantes) ---
data_path = Path('data/train.csv')
df = pd.read_csv(data_path)
df = df.drop_duplicates()

# 1. PADRONIZAÇÃO DE COLUNAS (O passo que causou o erro)
df.columns = df.columns.str.lower().str.replace(' ', '_')

# 2. LIMPEZA (Agora 'postal_code' existe)
df['postal_code'] = df['postal_code'].fillna(5401)
# (Assumindo 'dayfirst=True' do notebook original)
df['order_date'] = pd.to_datetime(df['order_date'], dayfirst=True) 

# 3. CRIAÇÃO DA VARIÁVEL ALVO ('demand')
product_demand = df.groupby('product_id')['sales'].sum().reset_index()

# (Etapa de qcut faltando no seu código, mas presente no original)
labels = ['very low', 'low', 'neutral', 'high', 'very high']
product_demand['demand'] = pd.qcut(product_demand['sales'], q=5, labels=labels)

df = pd.merge(
    left=df,
    right=product_demand[['product_id', 'demand']],
    on='product_id',
    how='left'
)

# 4. CRIAÇÃO DO TARGET NUMÉRICO ('demand_ord')
ordinal_mapping = {
    'very low': 0,
    'low': 1,
    'neutral': 2,
    'high': 3,
    'very high': 4
}
# Criamos no 'df' todo antes de fatiar
df['demand_ord'] = df['demand'].map(ordinal_mapping)
# (Tratamento de nulos caso o qcut ou merge falhem - o original não tinha, mas é boa prática)
df = df.dropna(subset=['demand_ord']) # Garante que o alvo não seja nulo
df['demand_ord'] = df['demand_ord'].astype('int8')


# 5. FEATURE ENGINEERING (Para 'ano', 'mes', 'dia_semana', etc.)
df['ano'] = df['order_date'].dt.year
df['mes'] = df['order_date'].dt.month
df['dia_semana'] = df['order_date'].dt.day_name(locale='pt_BR')
df['day_of_week'] = df['order_date'].dt.day_name() # Versão em inglês

# --- Divisão e Definição de X/y ---

df_treino = df[(df['order_date'] >= '2015-01-01') & (df['order_date'] <= '2017-12-31')].copy()

# 6. DEFINIÇÃO DE COLS_TO_DROP (Faltando no seu script)
# (Lista do notebook original, garantindo que 'product_id' saia)
cols_to_drop = [
    'demand', 'demand_ord', 'row_id', 'order_id', 'customer_name',
    'product_name', 'order_date', 'ship_date', 'product_id'
]

# 5. Definir seus dados de treino (Agora deve funcionar)
X_train = df_treino.drop(columns=cols_to_drop, errors='ignore')
y_train = df_treino['demand_ord']

print("X_train e y_train criados com sucesso.")
print(f"Shape do X_train: {X_train.shape}")

X_train e y_train criados com sucesso.
Shape do X_train: (6542, 15)


In [3]:
# 1. Definir os grupos de colunas
numeric_features = ['sales', 'postal_code', 'ano', 'mes'] # Adicione outras se houver
categorical_features = ['ship_mode', 'segment', 'city', 'state', 
                        'region', 'category', 'sub-category', 
                        'dia_semana', 'day_of_week']

# 2. Criar o transformador numérico
numeric_transformer = Pipeline(steps=[
    ('scaler', StandardScaler())
])

# 3. Criar o transformador categórico
categorical_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
])

# 4. Unir tudo com o ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ],
    remainder='drop'
)

# 5. Definir seus dados de treino
# (Usando o df_treino antes de qualquer processamento manual)
X_train = df_treino.drop(columns=cols_to_drop, errors='ignore')
y_train = df_treino['demand_ord']

In [4]:
class OrdinalRegressorWrapper(BaseEstimator, ClassifierMixin):
    """
    Wrapper que transforma um regressor em um classificador ordinal.
    Ele treina o regressor no target numérico (0-4) e, na predição,
    arredonda e limita (clip) o resultado para a classe inteira mais próxima.
    """
    def __init__(self, regressor, min_val=0, max_val=4):
        self.regressor = regressor
        self.min_val = min_val
        self.max_val = max_val

    def fit(self, X, y):
        # O 'regressor' (ex: XGBRegressor) é 'fitado' normalmente
        self.regressor.fit(X, y)
        return self

    def predict(self, X):
        # 1. Prever o valor float
        preds_float = self.regressor.predict(X)
        
        # 2. Arredondar para o inteiro mais próximo
        preds_rounded = np.round(preds_float)
        
        # 3. Limitar (clip) ao intervalo de classes [0, 4]
        preds_clipped = np.clip(preds_rounded, self.min_val, self.max_val)
        
        return preds_clipped.astype(int)

In [5]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
import lightgbm as lgb
import xgboost as xgb

# Dicionário de modelos para testar
models_to_compare = {
    'LogisticRegression_Multi': LogisticRegression(solver='liblinear', multi_class='ovr', random_state=42),
    'RandomForest_Multi': RandomForestClassifier(n_estimators=100, random_state=42),
    'LGBM_Multi': lgb.LGBMClassifier(objective='multiclass', num_class=5, random_state=42),
    'XGBoost_Reg': OrdinalRegressorWrapper(
        regressor=xgb.XGBRegressor(objective='reg:squarederror', 
                                   random_state=42, 
                                   n_estimators=100),
        min_val=0,
        max_val=4
    )
    
}

In [10]:
# Usamos 'neg_mean_absolute_error' pois o sklearn maximiza scores (erro negativo)
scoring_metrics = {
    'accuracy': 'accuracy',
    'mae': 'neg_mean_absolute_error'
}

# Lista para armazenar os resultados
cv_results = []

# Iterar sobre cada modelo no dicionário
print("Iniciando avaliação de múltiplos modelos...")
for model_name, model in models_to_compare.items():
    
    print(f"--- Avaliando: {model_name} ---")
    
    # 1. Criar o Pipeline completo: Preprocessor -> Model
    full_pipeline = Pipeline(steps=[
        ('preprocessor', preprocessor),
        ('model', model)
    ])
    
    # 2. Executar a Validação Cruzada (ex: 5 folds)
    # X_train e y_train são os dados BRUTOS (antes do pré-processamento)
    scores = cross_validate(
        full_pipeline, 
        X_train, 
        y_train, 
        cv=5, 
        scoring=scoring_metrics,
        n_jobs=-1
    )
    
    # 3. Armazenar os resultados médios
    cv_results.append({
        'Modelo': model_name,
        'Accuracy_Média': scores['test_accuracy'].mean(),
        'MAE_Média': -1 * scores['test_mae'].mean() # Corrigindo o sinal do MAE
    })

print("... Avaliação concluída.")

# 6. Consolidar e Apresentar Resultados
df_comparacao_cv = pd.DataFrame(cv_results).set_index('Modelo')

Iniciando avaliação de múltiplos modelos...
--- Avaliando: LogisticRegression_Multi ---
--- Avaliando: RandomForest_Multi ---
--- Avaliando: LGBM_Multi ---


/Users/jusbrasil/dg_dev/kfcaio-pmv-si-2025-2-pe7-t2-t1-g4/src/.venv/lib/python3.11/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
/Users/jusbrasil/dg_dev/kfcaio-pmv-si-2025-2-pe7-t2-t1-g4/src/.venv/lib/python3.11/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
/Users/jusbrasil/dg_dev/kfcaio-pmv-si-2025-2-pe7-t2-t1-g4/src/.venv/lib/python3.11/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
/Users/jusbrasil/dg_dev/kfcaio-pmv-si-2025-2-pe7-t2-t1-g4/src/.venv/lib/python3.11/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
/Users/jusbr

--- Avaliando: XGBoost_Reg ---
... Avaliação concluída.
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000190 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 757
[LightGBM] [Info] Number of data points in the train set: 5234, number of used features: 122
[LightGBM] [Info] Start training from score -1.940195
[LightGBM] [Info] Start training from score -1.573596
[LightGBM] [Info] Start training from score -1.619809
[LightGBM] [Info] Start training from score -1.578215
[LightGBM] [Info] Start training from score -1.407535
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000896 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 766
[LightGBM] [Info] Number of data points in the train set: 5234, number of used features: 127
[LightGBM] [Info] Start training f

In [7]:
# Exibir a tabela comparativa final
print("Tabela Comparativa de Métricas (Validação Cruzada, 5-Folds)")
display(df_comparacao_cv.style.format({
    'Accuracy_Média': '{:.4f}',
    'MAE_Média': '{:.4f}'
}).highlight_max(subset=['Accuracy_Média'], color='lightgreen')
  .highlight_min(subset=['MAE_Média'], color='lightgreen')
)

Tabela Comparativa de Métricas (Validação Cruzada, 5-Folds)


,Accuracy_Média,MAE_Média
Modelo,,
LogisticRegression_Multi,0.5136,0.6033
RandomForest_Multi,0.5402,0.5459
LGBM_Multi,0.5717,0.4859
XGBoost_Reg,0.5876,0.4482
